<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2016</h1>
<h1 style="font-size: 250%;">Problem Set #3</h1>
<h1 style="font-size: 200%;">Version 1.0</h1>
<h1>Issued Monday 10/03/2016; Due Monday 10/17/2016</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test your approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for each class meeting after their due date. Feel free to get help from others, but the work you submit in should be your own.

In [4]:
# Fill in the fields as described
class Student:
    first = "James"
    last = "Barbour"
    onyen = "jebarbou"
    pid = "720076615"

In this problem set we will revisit the *Monkepo* data discussed in Lecture 3. You may want to revist that lecture to reaquaint yourself with the data set. Your first task will be to construct and populate a database with the following schema.

 <table style="border:none;"><tbody>
        <tr style="border:none;">
            <td style="padding:0px 20px;border:none;" colspan="2">
            <table><tbody>
            <tr><th colspan="6" style="text-align:center;">Monkepo</th></tr>
            <tr><td><u>mid</u></td>
                <td>name</td>
            </tr>
            </tbody></table>
            </td>
            <td style="padding:20px 20px;border:none;">
            <table><tbody>
            <tr><th colspan="4" style="text-align:center;">Appearance</th></tr>
            <tr><td><u>mid</u></td>
                <td><u>latitude</u></td>
                <td><u>longitude</u></td>
                <td><u>datetime</u></td>
            </tr>
            </tbody></table>
            </td>
            <td style="padding:20px 20px;border:none;">
            <table><tbody>
            <tr><th colspan="4" style="text-align:center;">Class</th></tr>
            <tr><td><u>mid</u></td>
                <td><u>major (1/0)</u></td>
                <td>type</td>
            </tr>
            </tbody></table>
            </td>
        </tr>
        </tbody></table>

The *Monkepo* relation should hold just the *unique* name of a *monkepo* species and an autoincremented primary key, *mid*. When a *Monkepo* appears its location and appearance time are stored as a tuple in the *Appearance* relation. Finally, the *Class* relation is used to hold information about the major and minor class of the monkepo species. The major class is indicated by a value of *1* in the major field, and the minor class by a *0*. There should be no class entry for a *None* minor type.  

Use this verison of the <a href="http://csbio.unc.edu/mcmillan/Media/monkepo.csv" download>data file</a> to fill your database (it is different than the one used in lecture 3).

---
**Part 1:** In the cell provided below write a Python fragment to create the specified database, and name it *"monkepo.db"* (Hint: you should investigate Python's *DateTime* module to find methods for combining the date and time values from the data file into a single attribute). You code should commit and close your database.

In [11]:
import sqlite3, pandas, datetime, numpy
conn = sqlite3.connect('monkepo.db')
curs = conn.cursor()
curs.execute('CREATE TABLE Monkepo(\
             mid INTEGER PRIMARY KEY AUTOINCREMENT,\
             name TEXT,\
             UNIQUE(name))')
curs.execute('CREATE TABLE Appearance(\
             mid INTEGER,\
             latitude REAL,\
             longitude REAL,\
             datetime DATETIME,\
             PRIMARY KEY(mid, latitude, longitude, datetime),\
             FOREIGN KEY(mid) REFERENCES Monkepo(mid))')
curs.execute('CREATE TABLE Class(\
             mid INTEGER,\
             major BOOLEAN,\
             type TEXT,\
             PRIMARY KEY(mid, major),\
             FOREIGN KEY(mid) REFERENCES Monkepo(mid))')
monkcsv = pandas.read_csv('monkepo.csv')
monk_names = monkcsv.name.drop_duplicates()
monk_names.to_sql('Monkepo', conn, if_exists='append', index=False)


def converttime(row):
    try:
        return datetime.datetime.combine(
            datetime.datetime.strptime(row['date'], '%Y-%m-%d'),
            datetime.datetime.time(datetime.datetime.strptime(row['time'], '%H:%M:%S.%f')))
    except ValueError:
        return datetime.datetime.combine(
            datetime.datetime.strptime(row['date'], '%Y-%m-%d'),
            datetime.datetime.time(datetime.datetime.strptime(row['time'], '%H:%M:%S')))


def monkepotomid(row):
    return pandas.read_sql_query(
        'SELECT mid from Monkepo M WHERE M.name = "'+ row['name'] + '"', conn)['mid'][0]


monkcsv['mid'] = monkcsv.apply(monkepotomid, axis=1)
monk_sightings = monkcsv.drop(['majorclass', 'minorclass', 'date', 'time', 'name'], axis=1)
monk_sightings['datetime'] = monkcsv.apply(converttime, axis=1)
monk_sightings = monk_sightings.drop_duplicates()
monk_sightings.to_sql('Appearance', conn, if_exists='append', index=False,)
monk_class_major = monkcsv.drop(
    ['longitude', 'latitude', 'name', 'date', 'time', 'minorclass'], axis=1)
monk_class_minor = monkcsv.drop(
    ['longitude', 'latitude', 'name', 'date', 'time', 'majorclass'], axis=1)
monk_class_minor = monk_class_minor.drop_duplicates()
monk_class_major = monk_class_major.drop_duplicates()
monk_class_major.rename(columns={'majorclass':'type'}, inplace=True)
monk_class_minor.rename(columns={'minorclass':'type'}, inplace=True)
monk_class_major['major'] = True
monk_class_minor['major'] = False
monk_class = pandas.concat([monk_class_major, monk_class_minor])
monk_class = monk_class[monk_class.type != 'None']
monk_class.to_sql('Class', conn, if_exists='append', index=False)
conn.commit()
conn.close()

---
**Part 2:** Using the database that you created in part 1, write Python function to count the frequency of appearances of each Monkepo species. Your function should take the name of your database as an argument and return a sorted list of tuples (Monkepo.name, #appearances) with the most frequent first. Implement this function in using a dictionary like we did in Lecture 3.

In the cell below, enter the definition of your function, followed by a call to it, where the result is displayed via a Pandas data frame as shown in the code fragment supplied.

In [12]:
def appearancefreqpy(db):
    conn = sqlite3.connect(db)
    monks = pandas.read_sql_query('SELECT M.name\
                                  FROM (Monkepo JOIN Appearance USING(mid)) AS M',
                                  conn)
    conn.close()
    monkviews = {}
    for row in monks.values:
        monkviews[row[0]] = monkviews.get(row[0],0) + 1
    return sorted(monkviews.items(), key=lambda tup: tup[1], reverse=True)


pandas.DataFrame(appearancefreqpy('monkepo.db'), columns=['Name', 'Appearances'])

,Name,Appearances
0,PigDye,24445
1,AttaRat,18661
2,RopeSaw,10397
3,DewEel,8717
4,Veeee,7356
5,NovaNet,4766
6,TarPiece,4036
7,TubaZ,3639
8,ARasp,2593
9,AnIronAd-,2497


---
**Part 3:** Implement a second version of your function to count the frequency of appearances of each Monkepo species, but this time use SQL's "GROUP BY" directive. The input and output of this version should be the same as the one you implemented in Part 2. Display your answer using a Pandas data frame as well.

In [13]:
def appearancefreqsql(db):
    conn = sqlite3.connect(db)
    res = [tuple(tup) for tup in
            pandas.read_sql_query('SELECT M.name, count(M.mid)\
                                 FROM (Monkepo JOIN Appearance USING(mid)) AS M\
                                 GROUP BY(M.mid)\
                                 ORDER BY COUNT(M.mid) DESC',
                                 conn).values]
    conn.close()
    return res


pandas.DataFrame(appearancefreqsql('monkepo.db'), columns=['Name', 'Appearances'])

,Name,Appearances
0,PigDye,24445
1,AttaRat,18661
2,RopeSaw,10397
3,DewEel,8717
4,Veeee,7356
5,NovaNet,4766
6,TarPiece,4036
7,TubaZ,3639
8,ARasp,2593
9,AnIronAd-,2497


---
**Part 4:** Use Jupyter's [*%timeit*](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) function to compare the performance of the two functions that you wrote in parts 2 and 3.

In [14]:
%timeit[appearancefreqpy('monkepo.db')]
%timeit[appearancefreqsql('monkepo.db')]

10 loops, best of 3: 127 ms per loop
10 loops, best of 3: 20.5 ms per loop


---
**Part 5:** In the following cell write a Python function that finds both the rarest and most  common monkepo species for each class. Your function should take your database name as an input and it it should return a list with with a tuple for each distinct class type and columns with the most and least common Monkepo species names.

In [15]:
def maxminclass(db):
    conn = sqlite3.connect(db)
    classes = pandas.read_sql_query('SELECT DISTINCT type FROM Class', conn)
    res = []
    for i in classes['type']:
        maxmonk = pandas.read_sql_query('SELECT name\
                                 FROM (Monkepo JOIN Appearance USING(mid)\
                                       JOIN Class USING(mid)) AS M\
                                 WHERE type = "' + i + '"\
                                 GROUP BY M.mid\
                                 ORDER BY COUNT(M.mid) DESC\
                                 LIMIT 1',
                                 conn)
        minmonk = pandas.read_sql_query('SELECT name\
                                 FROM (Monkepo JOIN Appearance USING(mid)\
                                       JOIN Class USING(mid)) AS M\
                                 WHERE type = "' + i + '"\
                                 GROUP BY M.mid\
                                 ORDER BY COUNT(M.mid) ASC\
                                 LIMIT 1',
                                 conn)
        res.append((i, maxmonk.name[0], minmonk.name[0]))
    conn.close()
    return res
         
    
pandas.DataFrame(maxminclass('monkepo.db'), columns=['Class', 'Most Common', 'Least Common'])

,Class,Most Common,Least Common
0,Toxic,DewEel,RunSuave
1,Boring,PigDye,Asniper
2,Insect,DewEel,FerretTube
3,Haunting,Stylag,Nagger
4,Wet,Rustay,BasilToes
5,Turf,ARasp,RunSuave
6,Hot,TigerHowl,AlienSent
7,Mystic,GGExecute,Pyhon
8,Elfish,ClayFire,WiffyGlutG
9,Dirty,HerdSwans,XiNo


---
**Part 6:** One of the reasons that Monkepo appear is that they depend on supply stations in order to survive in Earth's harsh environment. The same smartphone app that Millennials use for detecting Mokepos also detects the locations of these supply stations. A list of detected stations is available for <a href="http://csbio.unc.edu/mcmillan/Media/stations.csv" download>download</a> here.

In the next cell you should add a new table to your database called *SupplyStations*. Make sure that you commit and close the modified database.

Then write a code fragment that finds, for each supply station how many Monkepo appearances were closer to it than to any other station. Your code should produce a list of tuples (stationID, latitude, longitude, #ofNearbyMonkepo) sorted from most popular to least popular.

***This problem is challenging, allow for enough time to do it!*** You can approximate the distance between a Monkepo appearance and a supply station by $\sqrt{(latitude_{Monkepo} - latitude_{Station})^2 + (longitude_{Monkepo} - longitude_{Station})^2}$. This is not techically correct, but it is close at the scales and latitudes we are considering.

In [17]:
conn = sqlite3.connect('monkepo.db')
curs = conn.cursor()
curs.execute('CREATE TABLE SupplyStations(\
             stationID INTEGER PRIMARY KEY,\
             latitude REAL,\
             longitude REAL)')
pandas.read_csv("stations.csv").to_sql('SupplyStations', conn, if_exists='append', index=False)
conn.commit()
conn.close()


def stationpopularity(db):
    appearances = pandas.read_sql_query('SELECT A.latitude, A.longitude\
                                        FROM Appearance A',
                                        sqlite3.connect(db))
    stations = pandas.read_sql_query('SELECT S.stationID, S.latitude, S.longitude\
                                            FROM SupplyStations S',
                                            sqlite3.connect(db))
    stations['count'] = 0
    for applat, applong in appearances.values:
        min_dist = 99999999999999999
        pos = 0
        station = 0
        for statid, statlat, statlong, count in stations.values:
            applat, applong = 0,1
            dist = numpy.sqrt(numpy.power(applat-statlat, 2)+numpy.power(applong-statlong, 2))
            if dist < min_dist:
                min_dist = dist
                station = pos
            pos += 1
        stations.set_value(station, 'count', stations['count'][station] + 1)
    stations.sort_values('count', axis=0, ascending=False, inplace=True, kind='quicksort')
    res = []
    for statid, statlat, statlong, count in stations.values:
        res.append((statid, statlat, statlong, count))
    return res
        

pandas.DataFrame(stationpopularity('monkepo.db'),
                 columns = ['Station', 'Longitude', 'Latitude', 'Nearby'])

,Station,Longitude,Latitude,Nearby
0,1947.0,35.770123,-78.631610,115942.0
1,1739.0,36.095616,-79.266743,0.0
2,1745.0,36.094323,-79.271895,0.0
3,1744.0,36.092072,-79.268503,0.0
4,1743.0,36.095298,-79.266995,0.0
5,1742.0,36.096445,-79.267359,0.0
6,1741.0,36.094837,-79.265162,0.0
7,1740.0,36.096056,-79.266388,0.0
8,1000.0,35.897835,-79.188710,0.0
9,1747.0,36.093651,-79.271198,0.0


Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set.

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. We do not need your database, since your Notebook should produce it.